In [ ]:
import os

os.chdir(r'C:\Users\carlo\GitHub\ProjectBuffalo')

In [ ]:
import buffalo.ingestion as ingestion
import buffalo.predictor as predictor

In [ ]:
import torch
import pandas as pd

In [ ]:
ingestor = ingestion.DataIngestion(ingestion.enum.API.ADVANTAGE)

In [ ]:
ingestor.load_data(r'cached_data/ingestion.sqlite')

In [ ]:
target_symbol = 'AAPL'

In [ ]:
ingestor.data['ADJUSTED_1M_STOCK'].symbol.unique()

In [ ]:
target_stock = ingestor.data['ADJUSTED_1M_STOCK'].query('symbol == @target_symbol')[['open', 'high', 'low', 'close', 'volume']]

In [ ]:
target_stock.dtypes

In [ ]:
other_stocks = ingestor.data['ADJUSTED_1M_STOCK'].query('symbol != "AAPL"')

In [ ]:
for symbol in ['MSFT', 'IBM']:
    temp = other_stocks[other_stocks['symbol'] == symbol][['open', 'high', 'low', 'volume']].rename(columns={'adjusted_close': 'close'})
    temp.columns = symbol + '_' + temp.columns
    target_stock = predictor.util.align_dataframe_by_time(target_stock, temp)

In [ ]:
target_income_statement = ingestor.data['COMPANY_INCOME_STATEMENT'].query('symbol == @target_symbol & freq == "quarterly"').drop(columns=['reported_currency', 'symbol', 'freq', 'function']).dropna(axis=1, how='all')
target_balance_sheet = ingestor.data['COMPANY_BALANCE_SHEET'].query('symbol == @target_symbol & freq == "quarterly"').drop(columns=['reported_currency', 'symbol', 'freq', 'function']).dropna(axis=1, how='all')
target_cash_flow = ingestor.data['COMPANY_CASH_FLOW'].query('symbol == @target_symbol & freq == "quarterly"').drop(columns=['reported_currency', 'symbol', 'freq', 'function', 'net_income']).dropna(axis=1, how='all')
fed_funds_rate = ingestor.data['FEDERAL_FUNDS_RATE'][['value']].rename(columns={'value': 'effective_federal_funds_rate'}).dropna(axis=1, how='all')
payroll = ingestor.data['NONFARM_PAYROLL'][['value']].rename(columns={'value': 'total_nonfarm_payroll'}).dropna(axis=1, how='all')
cpi = ingestor.data['CPI'][['value']].rename(columns={'value': 'consumer_price_index'}).dropna(axis=1, how='all')
unemployment = ingestor.data['UNEMPLOYMENT'][['value']].rename(columns={'value': 'unemployment_rate'}).dropna(axis=1, how='all')
real_gdp = ingestor.data['REAL_GDP'][['value']].rename(columns={'value': 'real_gross_domestic_product'}).dropna(axis=1, how='all')
real_gdp_per_capita = ingestor.data['REAL_GDP_PER_CAPITA'][['value']].rename(columns={'value': 'real_gross_domestic_product_per_capita'})
treasury_yield = ingestor.data['TREASURY_YIELD'][['value', 'maturity']].pivot(columns=['maturity'], values=['value']).dropna(axis=1, how='all')
treasury_yield.columns = 'treasury_yield_' + treasury_yield.columns.droplevel(level=0)

In [ ]:
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, target_income_statement)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, target_balance_sheet)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, target_cash_flow)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, fed_funds_rate)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, payroll)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, cpi)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, unemployment)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, real_gdp)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, real_gdp_per_capita)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, treasury_yield)
print(target_stock.shape)

In [ ]:
time_series_data = predictor.util.TimeSeries(endog=target_stock[['close']], exog=target_stock.drop(columns='close'), seq_len=20, batch_size=64, pin_memory=True, pin_memory_device='cuda')

In [ ]:
import buffalo.predictor.models as modeling

In [ ]:
rnn = modeling.RNN(input_size=99, hidden_size=64, num_layers=2, dropout=0.2, bidirectional=False)

In [ ]:
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001)

In [ ]:
loss_func = torch.nn.MSELoss()

In [ ]:
trainset_loader, testset_loader = time_series_data.get_traintest_splitted_dataloader(0.7, 64)

In [ ]:
modeling.train_model(rnn, optimizer, loss_func, trainset_loader, 0.1, epochs=10)

In [ ]:
target_stock